In [1]:
import cplex
import pandas as pd
import pandas_datareader as web
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
sp_49 = pd.read_csv('SP500_preparation.csv') # the dataframe got from the SP500_preparation.ipynb
names = sp_49.names # get the names of stocks in the stock pool

## Covariance matrix

In [6]:
# set starting and ending time

start = datetime.datetime(2016,10,1)
end = datetime.datetime(2017,10,1)

# make a dict: key is the name of stock, value is an array of monthly increasing rate
sp_50_dict = {}

In [53]:
sp_50_cov = np.zeros((49,49)) # covariance mat

In [54]:
# build covariance matrix
def calculate_increases(name, start, end, period):
    try:
        prices = web.DataReader(name, 'yahoo', start, end)
    except:
        return None
    
    Endings = prices.Close
    for i in range(len(Endings)):
        if Endings[i] is np.nan: Endings[i] = Endings[i-1]    
    increases = []
    for i in range(len(Endings) - period):
        increase = (Endings[i+period] - Endings[i])/Endings[i]
        increases.append(increase)
    increases_ = np.array(increases)
    return increases

for name in names:
    sp_50_dict[name] = calculate_increases(name, start, end, 20)

for i in range(49):
    for j in range(49):
        cov = np.cov(sp_50_dict[names[i]], sp_50_dict[names[j]])[0][1]
        sp_50_cov[i,j] = cov

In [55]:
sp_50_cov.to_csv('49StocksCovariance.csv')
sp_50_cov

array([[  1.37574584e-03,   6.72125365e-04,   3.29816509e-04, ...,
          3.73579948e-04,   8.06249450e-04,  -2.68508939e-04],
       [  6.72125365e-04,   1.21846822e-03,   5.11816612e-04, ...,
          7.49738675e-05,   5.58203128e-04,   2.93384055e-04],
       [  3.29816509e-04,   5.11816612e-04,   3.59680577e-03, ...,
          4.89127721e-04,   3.10055708e-04,   4.80534456e-04],
       ..., 
       [  3.73579948e-04,   7.49738675e-05,   4.89127721e-04, ...,
          8.29181071e-04,   3.64037953e-04,  -3.47438085e-04],
       [  8.06249450e-04,   5.58203128e-04,   3.10055708e-04, ...,
          3.64037953e-04,   1.00950920e-03,  -3.31596714e-04],
       [ -2.68508939e-04,   2.93384055e-04,   4.80534456e-04, ...,
         -3.47438085e-04,  -3.31596714e-04,   2.55120053e-03]])